In [1]:
import cosima_cookbook as cc
import numpy as np
import xarray as xr
import glob, sys
sys.path.append("/g/data/x77/ps7863/python_scripts")
import helper_functions as helper
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

import climtas.nci

from dask.distributed import Client


In [2]:
climtas.nci.GadiClient()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/35037/status,
Dashboard: /proxy/35037/status,Workers: 28
Total threads: 28,Total memory: 126.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35715,Workers: 28
Dashboard: /proxy/35037/status,Total threads: 28
Started: Just now,Total memory: 126.00 GiB
Comm: tcp://127.0.0.1:39701,Total threads: 1
Dashboard: /proxy/33741/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:36265,


2024-12-20 10:15:36,618 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:39135'.
2024-12-20 10:15:36,620 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:38797'.
2024-12-20 10:15:36,621 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:37109'.
2024-12-20 10:15:36,622 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:41989'.
2024-12-20 10:15:36,623 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:44849'.
2024-12-20 10:15:36,625 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:35847'.
2024-12-20 10:15:36,626 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:45747'.
2024-12-20 10:15:36,628 - distributed.scheduler - WARNING - Received heartbeat from unregistered 

In [3]:
session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
iaf_cycle3 = '01deg_jra55v140_iaf_cycle3'
expt = '01deg_jra55v140_iaf_cycle3_antarctic_tracers'

In [4]:
first_year = '1958'
last_year = '1960'
start_time=first_year+'-08-01'
end_time=last_year+'-08-31'
time_slice = slice(start_time, end_time)


In [5]:
save_dir = '/g/data/x77/ps7863/data/AABW_variability'


In [6]:
lon_slice = slice(80-360, 190-360)
lat_slice = slice(-82, -55)

In [7]:
tr_adelie = cc.querying.getvar(iaf_cycle3, 'passive_adelie', session,
                               frequency='1 monthly',
                               attrs={'cell_methods': 'time: mean'},
                               start_time=start_time, end_time=end_time,
                               chunks={'time':12, 'xt_ocean':100, 'yt_ocean':100})

tr_adelie = tr_adelie.sel(time=slice(start_time, end_time)).sel(yt_ocean=lat_slice).sel(xt_ocean=lon_slice)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "yt_ocean" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "xt_ocean" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "yt_ocean" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3

In [8]:
tr_ross = cc.querying.getvar(iaf_cycle3, 'passive_ross',
                            session,
                             frequency='1 monthly',
                             attrs={'cell_methods': 'time: mean'},
                             start_time=start_time, end_time=end_time,
                             chunks={'time':12, 'xt_ocean':100, 'yt_ocean':100})

tr_ross = tr_ross.sel(time=slice(start_time, end_time)).sel(yt_ocean=lat_slice).sel(xt_ocean=lon_slice)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "yt_ocean" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "xt_ocean" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "yt_ocean" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3

In [9]:
bot_mask = tr_ross.where(~np.isfinite(tr_ross.shift({'st_ocean': -1})))
bot_mask = ~np.isnan(bot_mask)
bot_mask

<xarray.DataArray 'passive_ross' (time: 25, st_ocean: 75, yt_ocean: 584,
                                  xt_ocean: 1100)> Size: 1GB
dask.array<invert, shape=(25, 75, 584, 1100), dtype=bool, chunksize=(3, 19, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 9kB -279.9 -279.8 -279.7 ... -170.1 -170.0
  * yt_ocean  (yt_ocean) float64 5kB -81.11 -81.07 -81.02 ... -55.07 -55.01
  * st_ocean  (st_ocean) float64 600B 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) datetime64[ns] 200B 1958-08-16T12:00:00 ... 1960-08-16T1...
Attributes:
    long_name:      passive (ross)
    units:          dimensionless
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    ncfiles:        ['/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_...
    contact:        Andrew Kiss
    email:          andrew.kiss@anu.edu.au
    created:        2020-11-11
    description:    0.1 degree ACCESS-OM2 global model configuration under in...
    notes:          Run configuration and history: https://github.com/COSIMA/...

In [10]:
bottom_depth = tr_ross.st_ocean.where(bot_mask).max(dim='st_ocean')
depth_threshold = bottom_depth - 500
mask_bottom_500 = tr_ross.st_ocean >= depth_threshold
mask_bottom_500 = tr_ross.where(mask_bottom_500)
mask_bottom_500 = ~np.isnan(mask_bottom_500)
mask_bottom_500

In [51]:
mask_bottom_500.mean('time').to_dataset(name='Mask for bottom 500 m').to_netcdf(f'{save_dir}/mask_bottom_500m.nc')

In [11]:
dzt = cc.querying.getvar(iaf_cycle3, 'dzt', session,
                               frequency='1 monthly',
                               start_time=start_time, end_time=end_time,
                               chunks={'time':12, 'xt_ocean':100, 'yt_ocean':100})

dzt = dzt.sel(time=slice(start_time, end_time)).sel(yt_ocean=lat_slice).sel(xt_ocean=lon_slice)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "yt_ocean" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "xt_ocean" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "yt_ocean" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3

In [12]:
dzt_bottom_500 = dzt.where(mask_bottom_500) 

In [13]:
adelie_bottom_500 = tr_adelie.where(mask_bottom_500)
# ross_bottom_500 = tr_ross.where(mask_bottom_500)

In [43]:
adelie_bottom_500_mean = (adelie_bottom_500 * dzt_bottom_500).sum('st_ocean') / dzt_bottom_500.sum('st_ocean').load()
# ross_bottom_500_mean = (ross_bottom_500 * dzt_bottom_500).sum('st_ocean') / dzt_bottom_500.sum('st_ocean')

In [44]:
adelie_bottom_500_mean

<xarray.DataArray (time: 25, yt_ocean: 584, xt_ocean: 1100)> Size: 64MB
dask.array<truediv, shape=(25, 584, 1100), dtype=float32, chunksize=(3, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 9kB -279.9 -279.8 -279.7 ... -170.1 -170.0
  * yt_ocean  (yt_ocean) float64 5kB -81.11 -81.07 -81.02 ... -55.07 -55.01
  * time      (time) datetime64[ns] 200B 1958-08-16T12:00:00 ... 1960-08-16T1...

In [45]:
adelie_climatology = adelie_bottom_500_mean.groupby('time.month').mean('time')
adelie_climatology

INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: method = None, returning None
INFO:flox:_choose_engine: Choosing 'numpy'
INFO:flox:find_group_cohorts: merging cohorts
INFO:flox:_choose_method: method is None
INFO:flox:_choose_method: choosing preferred_method=cohorts
INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: reindex is False


<xarray.DataArray (month: 12, yt_ocean: 584, xt_ocean: 1100)> Size: 31MB
dask.array<transpose, shape=(12, 584, 1100), dtype=float32, chunksize=(3, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 9kB -279.9 -279.8 -279.7 ... -170.1 -170.0
  * yt_ocean  (yt_ocean) float64 5kB -81.11 -81.07 -81.02 ... -55.07 -55.01
  * month     (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

In [46]:
peak_month = (adelie_climatology.argmax('month')+1).load()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
 

ValueError: All NaN slice encountered

2024-12-20 10:18:37,860 - distributed.worker - WARNING - Compute Failed
Key:       ('nanarg_agg-aggregate-add-6126034210ff29f0e3fe098f94c9ac04', 2, 9)
State:     executing
Function:  execute_task
args:      ((subgraph_callable-e2a5d0fe3ad8a372d8fe50bfac746700, (Compose(functools.partial(<function nanarg_agg at 0x14b08070d360>, <function _nanargmax at 0x14b08070d510>, axis=(0,), keepdims=False), functools.partial(<function _concatenate2 at 0x14b0806572e0>, axes=[0])), [array([[[(           nan, 0), ( 3.5282046e-15, 1), ( 3.3394009e-15, 1),
         ..., ( 6.0831593e-16, 2), ( 1.7045569e-18, 0),
         ( 2.0471516e-15, 2)],
        [(           nan, 0), ( 2.4243841e-18, 1), ( 2.3534451e-17, 1),
         ..., ( 1.3736975e-15, 2), (-2.7690940e-18, 0),
         ( 2.2324146e-15, 2)],
        [(           nan, 0), (           nan, 0), (           nan, 0),
         ..., ( 4.7583177e-16, 2), (-1.0153010e-17, 0),
         ( 2.3223863e-15, 2)],
        ...,
        [( 3.9076217e-10, 2), ( 3.341

In [48]:
peak_month.plot()

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-2629476' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:2391> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py", line 2400, in wait
    raise AllExit()
distributed.client.AllExit
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-2629477' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:2391> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py", line 2400, in wait
    raise AllExit()
distributed.client.AllExit
ERRO

ValueError: All NaN slice encountered

'vals', '<f4'), ('arg', '<i8')]), array([[[(nan, 3), (nan, 3), (nan, 3), ..., (nan, 3), (nan, 3),
         (nan, 3)],
        [(nan, 3), (nan, 3), (nan, 3), ..., (nan, 3), (nan, 3),
         (nan, 3)],
     
kwargs:    {}
Exception: "ValueError('All NaN slice encountered')"
Traceback: '  File "cytoolz/functoolz.pyx", line 518, in cytoolz.functoolz.Compose.__call__\n  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/array/reductions.py", line 1167, in nanarg_agg\n    raise ValueError("All NaN slice encountered")\n'

2024-12-20 10:20:03,861 - distributed.worker - WARNING - Compute Failed
Key:       ('nanarg_agg-aggregate-add-6126034210ff29f0e3fe098f94c9ac04', 3, 2)
State:     executing
Function:  execute_task
args:      ((subgraph_callable-e2a5d0fe3ad8a372d8fe50bfac746700, (Compose(functools.partial(<function nanarg_agg at 0x1525f5ba24d0>, <function _nanargmax at 0x1525f5ba2680>, axis=(0,), keepdims=False), functools.partial(<function _conca

In [33]:
# adelie_bottom_500_mean=adelie_bottom_500_mean.assign_attrs(Description='ALBW tracer averaged over bottom 500-m layer')
adelie_bottom_500_mean.encoding.update({'_FillValue': None})
adelie_bottom_500_mean.to_netcdf(f'{save_dir}/ALBW_tracer_bottom_500_{first_year}-{last_year}.nc')


/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
 

In [39]:
albw_tracer = xr.open_dataset('/g/data/x77/ps7863/data/AABW_variability/ALBW_tracer_bottom_500_1958-1960.nc')
# rsbw_tracer = xr.open_dataset('/g/data/x77/ps7863/data/AABW_variability/RSBW_tracer_bottom_500_1958-1960.nc')
albw_tracer
albw_tracer = albw_tracer['__xarray_dataarray_variable__'].load()
# rsbw_tracer = rsbw_tracer['RSBW tracer']

In [40]:
albw_climatology = albw_tracer.groupby('time.month').mean('time')
# rsbw_climatology = rsbw_tracer.groupby('time.month').mean('time')

INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: method = None, returning None
INFO:flox:_choose_engine: Choosing 'numpy'


In [41]:
albw_peak_month = albw_climatology.argmax('month')+1
# rsbw_peak_month = rsbw_climatology.argmax('month')+1

ValueError: All-NaN slice encountered